# Imports

In [1]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

import node_models
import loader
import training
import metrics
import autotune
import config

# GPU

In [2]:
device = torch.device("cuda")

# Data Loading

In [3]:
mnist_data_path = './train-images-idx3-ubyte'
mnist_label_path = './train-labels-idx1-ubyte'

In [4]:
#5-fold generation
mnist_data = loader.MNIST(mnist_data_path, mnist_label_path, 5)
mnist_splits = mnist_data.splits

In [5]:
print("MNIST STATS")
print("Number of splits:", len(mnist_splits))
print("Number of segments per split (train, val, test):", len(mnist_splits[0]))
print("Info per segment (data, labels):", len(mnist_splits[0][0]))
print("Size of segement (num examples):", len(mnist_splits[0][0][0]))

MNIST STATS
Number of splits: 5
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 48000


# Model & Optimizer

In [6]:
model = node_models.NeuralODE
optimizer = optim.Adam

# Training and Evaluation

## MNIST

In [7]:
model_params = config.config_mnist_anode['model']
lr, epochs, batch, workers = config.config_mnist_anode['train']

In [8]:
mnist_trainer = training.Trainer(model, optimizer, mnist_data, device)

In [ ]:
mnist_trainer.train(model_params, lr, epochs, batch, workers, verbose=True, num_loss=10)

[1,   150] loss: 16.64143
[1,   300] loss: 2.10376
[1,   450] loss: 0.86781
[1,   600] loss: 0.50821
[1,   750] loss: 0.38545
[1,   900] loss: 0.37297
[1,  1050] loss: 0.33881
[1,  1200] loss: 0.29392
[1,  1350] loss: 0.35654
[1,  1500] loss: 0.41418
[Fold 1] Epoch:1 Training Acc:0.8763958333333334


In [ ]:
mnist_trainer.test(model_params, 128, 12)

In [ ]:
mnist_plotter = metrics.Plotter(mnist_trainer.val_metrics)

In [ ]:
mnist_plotter.plotLoss("Loss")
mnist_plotter.plotAccuracy("Acc")

# Hyperparam Search

In [ ]:
from torch import nn
class TuneNN(nn.Module):
    def __init__(self, in_channels, hidden_dims):
        super(TuneNN, self).__init__()

        self.blockOne = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)
        )
        
        self.blockTwo = nn.Sequential(
            nn.Conv2d(hidden_dims, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)
        )
        
        self.blockThree = nn.Sequential(
            nn.Conv2d(hidden_dims, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.1),
        )
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        self.linear = nn.Sequential(
            nn.Linear(hidden_dims, 10),
        )
            
    def forward(self, x):
        x = self.blockOne(x)
        x = self.blockTwo(x)
        x = self.blockThree(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [ ]:
# model params - *args as tuple (model based)
# train params - *args as tuple (learning rate, epochs, batch size, number of workers)

config_1 = {
    'model' : (1,8),
    'train' : (1e-2, 1, 32, 8)
}

config_2 = {
    'model' : (1,32),
    'train' : (1e-3, 1, 32, 8)
}

config_3 = {
    'model' : (1,64),
    'train' : (1e-3, 1, 32, 8)
}

configs = [config_3, config_2, config_1]

In [ ]:
hyperparam = autotune.Autotuner(TuneNN, optimizer, mnist_data, device)

In [ ]:
hyperparam.search(configs)

In [ ]:
hyperparam.bestConfig()